# Illustration of w-snapshots (using w-stacking)

In [ ]:
%matplotlib inline

import sys
sys.path.append('../..')

from matplotlib import pylab as plt

import itertools
import numpy
import numpy.linalg
import scipy
import scipy.special
import time

from crocodile.clean import *
from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from arl.test_support import create_named_configuration, export_visibility_to_hdf5
from arl.data_models import *

Generate baseline coordinates for an observation with the VLA over 6 hours, with a visibility recorded every 10 minutes. The phase center is fixed at a declination of 45 degrees. We assume that the imaged sky says at that position over the course of the observation.

Note how this gives rise to fairly large $w$-values.

In [ ]:
vlas = create_named_configuration('VLAA')
ha_range = numpy.arange(numpy.radians(0),
                        numpy.radians(90),
                        numpy.radians(90 / 36))
dec = numpy.radians(45)
vobs = xyz_to_baselines(vlas.data['xyz'], ha_range, dec)
wvl = 5
uvw_in = vobs / wvl

## General parameters

In [ ]:
# Imaging parameterisation
theta = 2*0.05
lam = 18000
wstep = 100
npixkern = 61

# Scale for kernel size? This will make kernel size predictable, but reduce our
# field of view (and therefore decrease image sharpness).
scaleByDet = False

grid_size = int(numpy.ceil(theta*lam))
print("Grid size: %dx%d" % (grid_size, grid_size))

# Determine Transformation

Now we assume that we want to shift the image centre somewhere else. We transform the image at the same time in a way that optimises w-shape:

In [ ]:
dl = 0.67
dm = 0.67
dn = numpy.sqrt(1 - dl**2 - dm**2)

print("Elevation: %.f deg (if phase centre is zenith)\n" % \
      numpy.rad2deg(numpy.arcsin(dn)))

T = kernel_transform(dl, dm)
Tdet = (numpy.sqrt(numpy.linalg.det(T)) if scaleByDet else 1)

T *= Tdet
print("Transformation:\n %s [l, m]  + [%f, %f]" % (T, dl, dm))
print("Determinant: ", numpy.linalg.det(T))

Visualise where our transformed field-of-view sits in the original `lm`-space:

In [ ]:
plt.rcParams['figure.figsize'] = 8, 8
ax = plt.subplot(111)
coords = 0.01 * numpy.transpose(numpy.meshgrid(range(-3, 4), range(-3, 4)))
facet_centres = numpy.sin(2 * numpy.pi * 0.025 *
                          numpy.vstack(numpy.transpose(numpy.meshgrid(range(-10, 11), range(-10, 11)))))
for dp in facet_centres:
    if dp[0]**2+dp[1]**2 >= 1: continue
    Tx = kernel_transform(*dp)
    if scaleByDet: Tx *= numpy.sqrt(numpy.linalg.det(Tx))
    xys = numpy.dot(coords, Tx) + dp
    plt.scatter(*numpy.transpose(xys),c='gray')
xys = numpy.dot(coords, T) + numpy.array([dl, dm])
plt.scatter(*numpy.transpose(xys),c='red')
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_xlabel('l')
ax.set_xlabel('m')
plt.show()

## Create visibilities

We place dots where the transformed field of view is going to end up at

In [ ]:
vis_in = numpy.zeros(len(uvw_in), dtype=complex)
dp = numpy.array([dl, dm])
for coord in itertools.product(range(-3, 4), range(-3, 4)):
    p = numpy.dot(0.01*numpy.array(coord), T) + dp
    vis_in += simulate_point(uvw_in, *p)
# Extra dot to mark upper-right corner
p = numpy.dot(T, numpy.array([0.028, 0.028])) + dp
vis_in += simulate_point(uvw_in, *p)
# Extra dot to mark upper-left corner
p = numpy.dot(T, numpy.array([-0.032, 0.028])) + dp
vis_in += simulate_point(uvw_in, *p)

plt.rcParams['figure.figsize'] = 16, 8
plt.clf()
uvdist=numpy.sqrt(uvw_in[:,0]**2+uvw_in[:,1]**2)
plt.plot(uvdist, numpy.abs(vis_in), '.', color='r')

Using imaging, we can now reconstruct the image. We split the visibilities into a number of w-bins:

In [ ]:
# Determine weights (globally)
wt = doweight(theta, lam, uvw_in, numpy.ones(len(uvw_in)))

# Depending on algorithm we are going to prefer different uvw-distributions,
# so make decision about conjugation of visibilities flexible.
def flip_conj(where):
    # Conjugate visibility. This does not change its meaning.
    uvw = numpy.array(uvw_in)
    vis = numpy.array(vis_in)
    uvw[where] = -uvw[where]
    vis[where] = numpy.conj(vis[where])
    # Determine w-planes
    wplane = numpy.around(uvw[:,2] / wstep).astype(int)
    return uvw, vis, numpy.arange(numpy.min(wplane), numpy.max(wplane)+1), wplane

## Prepare for imaging

First apply the image-space linear transformation by applying the inverse in visibility space. Then create the w-kernel and apply re-centering so kernels to not go out of bounds.

In [ ]:
uvw,vis,wplanes,wplane = flip_conj(uvw_in[:,1] < 0.0)

# Apply visibility transformations (l' m') = T (l m) + (dl dm)
vis = visibility_shift(uvw, vis, -dl,-dm)
uvw = uvw_transform(uvw, numpy.linalg.inv(T))

# Choose width of UV-towers. Best way is to choose it so we get a nice 2^x size below
uvbin_size = 256 - npixkern

# Generate Fresnel pattern for shifting between two w-planes
# As this is the same between all w-planes, we can share it
# between the whole loop.
l,m = kernel_coordinates(uvbin_size + npixkern, theta, T=T, dl=dl, dm=dm)
wkern = w_kernel_function(l, m, wstep)

# Center kernels by moving the grid pattern into one direction and adding the opposite offset to visibilities
wkern = kernel_recentre(wkern, theta, wstep, dl*Tdet, dm*Tdet)
uvw = visibility_recentre(uvw, dl*Tdet, dm*Tdet)

# Check kernel in grid space at maximum w to make sure that we managed to center it
plt.rcParams['figure.figsize'] = 16, 8
show_grid(ifft(wkern**(numpy.max(uvw[:,2])/wstep)), "wkern", theta)
show_grid(ifft(wkern**(numpy.min(uvw[:,2])/wstep)), "wkern", theta)

In [ ]:
start_time = time.time()
grid_sum = numpy.zeros((grid_size, grid_size), dtype=complex)
ubin = numpy.floor(uvw[:,0]*theta/uvbin_size).astype(int)
vbin = numpy.floor(uvw[:,1]*theta/uvbin_size).astype(int)

wkern_shifted = numpy.fft.fftshift(wkern)
src = numpy.ndarray((len(vis), 0))
for ub in range(numpy.min(ubin), numpy.max(ubin)+1):
    for vb in range(numpy.min(vbin), numpy.max(vbin)+1):
        
        # Find visibilities
        bin_sel = numpy.logical_and(ubin == ub, vbin == vb)
        if not numpy.any(bin_sel):
            continue
        
        # Determine bin dimensions
        xy_min = uvbin_size * numpy.array([ub, vb], dtype=int)
        xy_max = uvbin_size * numpy.array([ub+1, vb+1], dtype=int)
        uv_min = xy_min / theta
        uv_max = xy_max / theta
        uv_mid = (xy_max + xy_min) // 2 / theta

        # Make sure we have enough space for convolution.
        xy_min -= (npixkern + 1) // 2
        xy_max += npixkern // 2
        assert(numpy.all(numpy.max(xy_max - xy_min) == uvbin_size+npixkern))
        uvw_size = (uvbin_size+npixkern) / theta

        # Make grid for uv-bin
        bin_image_sum = numpy.zeros((uvbin_size+npixkern, uvbin_size+npixkern), dtype=complex)
        nvis = 0; midws = []
        last_wp = wplanes[0]
        for wp in wplanes:

            # Filter out visibilities for u/v-bin and w-plane
            slc = numpy.logical_and(bin_sel, wplane == wp)
            puvw = uvw[slc]
            if len(puvw) == 0: continue
            pvis = vis[slc]
            pwt = wt[slc]
            
            # Statistics
            nvis += len(puvw)
            midws.append(wp*wstep)
            
            # w=0 plane? Just grid directly
            if wp == 0:
                ivis = pvis * pwt / w_kernel_function(dl, dm, puvw[:,2])
                grid_sum += simple_imaging(theta, lam, puvw, src, pvis * pwt)
                continue

            # Bring image sum into this w-plane
            if last_wp != wplanes[0]:
                bin_image_sum *= wkern_shifted**(wp-last_wp)
            last_wp = wp
            
            # Grid relative to mid-point
            uvw_mid = numpy.hstack([uv_mid, [wp*wstep]])
            puvw_new = puvw - uvw_mid
            ivis = pvis * pwt / w_kernel_function(dl, dm, puvw_new[:,2])
            pgrid = simple_imaging(theta, uvw_size, puvw_new, src, ivis)
            
            # Add to bin grid
            bin_image_sum += numpy.fft.ifft2(pgrid)

        # No visibilities? Skip
        if nvis == 0: continue

        # Transfer into w=0 plane, FFT image sum
        bin_image_sum /= wkern_shifted**last_wp
        bin_grid = numpy.fft.fft2(bin_image_sum)

        # Add to grid, keeping bounds in mind
        mid = int(lam*theta)//2
        x0, y0 = mid + xy_min
        x1, y1 = mid + xy_max
        x0b, y0b = numpy.amax([[x0, y0], [0,0]], axis=0)
        x1b, y1b = numpy.amin([[x1, y1], [grid_size,grid_size]], axis=0)
        grid_sum[y0b:y1b, x0b:x1b] += \
           bin_grid[y0b-y0:y1b-y0, x0b-x0:x1b-x0]

plt.rcParams['figure.figsize'] = 16, 12
print("Done in %.1fs" % (time.time() - start_time))
new_image_sum = ifft(grid_sum)
show_image(2.0 * numpy.real(new_image_sum), "image", theta)

## Zoom in

In [ ]:
image_show = 2 * numpy.real(new_image_sum)
step=int(grid_size/10)
def zoom(x, y=step): plt.matshow(image_show[y:y+2*step,x:x+2*step]) ; plt.colorbar(shrink=.4,pad=0.025);  plt.show()
from ipywidgets import interact
interact(zoom, x=(0,image_show.shape[0]-2*step,step), y=(0,image_show.shape[1]-2*step,step));

## Kernels used depending on parameters

In [ ]:
import numpy.linalg
def show_kernel(w=6000, dl=0.60, dm=0.60, scale=scaleByDet):
    size = 256
    T = kernel_transform(dl, dm)
    Tdet = (numpy.sqrt(numpy.linalg.det(T)) if scale else 1)
    T *= Tdet
    print("Determinant T: ", numpy.linalg.det(T))
    l,m = kernel_coordinates(size, theta, dl=dl, dm=dm, T=T)
    wkern = w_kernel_function(l, m, w)
    wkern = kernel_recentre(wkern, theta, w, dl*Tdet, dm*Tdet)
    show_grid(ifft(wkern), "wkern_{w=%.0f,dl=%.2f,dm=%.2f}" % (w,dl,dm), theta)
interact(show_kernel,
         w=(numpy.min(uvw[:,2]), numpy.max(uvw[:,2])),
         dl=(-.99,.99,0.01), dm=(-.99,.99,0.01));

## Behaviour of kernel close to w=0

Note that with shifts the kernel starts oscillating much more quickly as $w$ changes. This can easily be demonstrated by looking at the centre pixel of kernels with low $w$. This can be approximated by evaluating the $w$-function itself at $(dl, dm)$.

In [ ]:
def show_kernel0(dl2=0.60, dm2=0.60, scale=scaleByDet):
    T = kernel_transform(dl2, dm2)
    Tdet = (numpy.sqrt(numpy.linalg.det(T)) if scale else 1)
    T *= Tdet
    size = 7
    vals = []
    approx = []
    ws = numpy.arange(10*wstep) / 10
    l,m = kernel_coordinates(size, theta, dl=dl2, dm=dm2, T=T)
    for w in ws:
        wkern = kernel_recentre(w_kernel_function(l, m, w), theta, w, dl2*Tdet, dm2*Tdet)
        vals.append(ifft(wkern)[size//2,size//2])
        approx.append(w_kernel_function(dl2, dm2, w))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(ws, numpy.real(vals), label="value")
    ax.plot(ws, numpy.real(approx), lw=.5, label="approximation")
    ax.set_xlabel("w")
    ax.set_ylabel("G~(0,0,w)")
    plt.legend()
    plt.show()
interact(show_kernel0, dl2=(-.99,.99,0.01), dm2=(-.99,.99,0.01));    

## Export visibilities to HDF5

In [ ]:
import astropy
import astropy.units as u
import itertools
bl_ants = numpy.array(list(itertools.combinations(range(len(vlas.xyz)), 2)))
bl_count = bl_ants.shape[0]
vlas.name = "VLAS"
vis_obj = Visibility(
    frequency=[299792458 / wvl],
    phasecentre = SkyCoord(ra=0*u.rad, dec=dec*u.degree),
    configuration = vlas,
    uvw = vobs,
    time = numpy.repeat(ha_range/numpy.radians(360), bl_count),
    vis = vis_in.reshape((len(vis_in),1,1)),
    antenna1 = numpy.tile(bl_ants[:,1], len(ha_range)),
    antenna2 = numpy.tile(bl_ants[:,0], len(ha_range)),
    weight = numpy.ones_like(vis_in)
)
import h5py
f = h5py.File('test.h5','w')
export_visibility_to_hdf5(vis_obj, f, 'vis')
f.close()